In [ ]:
import os
import glob
import xarray as xr
#import WAPORWA modules
os.chdir(r'D:\Github\WAPORWA\modules') #change to modules path
import WaPOR
import WA
from WA.pickle_basin import pickle_in,pickle_out  

#Read pickle
pickle=glob.glob(r'E:\WA_Sheet1\Main\Niger\*.pickle')[-1]
BASIN=pickle_in(pickle)  

In [ ]:
time='A-{0}'.format(BASIN['end_month'])
for key in ['p','et','etincr','etrain']:
    nc=Basin['main_data']['monthly'][key]
    var,name=open_nc(nc)
    var_y=var.resample(time=time).sum(dim='time',skipna=False)
    outfolder=os.path.join(Basin['Dir'],'data','nc')  
    attrs={"units":"mm/year", "source": "Hydrolological year ", "quantity":name}
    var_y.assign_attrs(attrs)
    var_y.name = name
    var_y_dts=var_y.chunk({"latitude":-1,"longitude":-1}).to_dataset()
    nc_fn="{0}_hyearly1.nc".format(key)
    nc_path=os.path.join(outfolder,nc_fn)
    var_y_dts.to_netcdf(nc_path,encoding={name:{'zlib':True}})
    Basin['main_data']['yearly'][key]=nc_path

In [ ]:
area_fh=Basin['main_data']['stat']['area']
# area_fh=Basin['main_data']['subbasin']['Zarqa']
area=gis.OpenAsArray(area_fh,nan_values=True)